<a href="https://colab.research.google.com/github/wildmary/TEI-encoded-Dictionary/blob/main/%D1%81%D0%BB%D0%BE%D0%B2%D0%B0%D1%80%D1%8C_TEI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import re 
import csv
from lxml import etree
import pandas as pd

In [33]:
def init():
  slovar = pd.read_csv("whole_dict_edited.csv", sep=";")
  slovar.columns = ["Griechisch", "GLemma", "GMSA", "Slavisch", "SÜbersetzung", "SemantK", "Lateinisch", "LLemma", "LGram"]
  root = etree.Element("TEI", xmlns="http://www.tei-c.org/ns/1.0")
  teiheader = etree.Element("teiHeader")
  root.append(teiheader)
  fileDesc = etree.SubElement(teiheader, "fileDesc")
  titleStmt = etree.SubElement(fileDesc, "titleStmt")
  title = etree.SubElement(titleStmt, "title")
  title.text = "Словарь Епифания Славинецкого"
  text = etree.SubElement(root, "text")
  body = etree.SubElement(text, "body")

In [114]:
#def generate_xml():
slovar = pd.read_csv("whole_dict_edited.csv", sep=";")
slovar.columns = ["Griechisch", "GLemma", "GMSA", "Slavisch", "SÜbersetzung", "SemantK", "Lateinisch", "LLemma", "LGram"]
root = etree.Element("TEI", xmlns="http://www.tei-c.org/ns/1.0")
teiheader = etree.Element("teiHeader")
root.append(teiheader)
fileDesc = etree.SubElement(teiheader, "fileDesc")
titleStmt = etree.SubElement(fileDesc, "titleStmt")
title = etree.SubElement(titleStmt, "title")
title.text = "Словарь Епифания Славинецкого"
text = etree.SubElement(root, "text")
body = etree.SubElement(text, "body")
for index, row in slovar.iterrows():
  if index == 0:
    continue
  else:
    if type(row["Griechisch"]) != float:
      if type(row["GLemma"]) == float and type(row["SÜbersetzung"]) == float and type(row["LLemma"]) == float:
        page_note(row)
      if type(row["GLemma"]) != float:
        entry = etree.SubElement(body, "entry", xml_id=row["GLemma"], xml_lang="grc")
        form = etree.SubElement(entry, "form", type="hyperlemma")
        orth = etree.SubElement(form, "orth")
        orth.text = row["Griechisch"]
      else:
        continue
    if type(row["GLemma"]) != float:
      if type(row["Griechisch"]) == float:
        entry = etree.SubElement(body, "entry", xml_id=row["GLemma"], xml_lang="grc")
      form = etree.SubElement(entry, "form", type="lemma")
      orth = etree.SubElement(form, "orth")
      orth.text = row["GLemma"]
      if type(row["GMSA"]) != float: 
        grammar(row["GMSA"])
      if type(row["SÜbersetzung"]) != float:
        sense_greek = etree.SubElement(entry, "sense", xml_id=row["GLemma"]+".1")
        slavic_latin_translation()
        several_slavic_equivalents_check(slovar["SÜbersetzung"], slovar["Lateinisch"], slovar["LLemma"])
    if type(row["SemantK"]) != float:
      semantischer_kommentar()
etree.ElementTree(root).write('dictionary_TEI.xml', pretty_print=True, xml_declaration = True, encoding='UTF-8', standalone = None)

In [48]:
def page_note(row):
  column = re.search("^(?P<f>f.)\s(?P<page_num>\d\d\d.).*(?P<side>linke|rechte)\s(?P<Spalte>Spalte)", row["Griechisch"])
  page = re.search("Kustode.*(?P<pg>\d\d\d.)", row["Griechisch"])
  if column:
    cb = etree.SubElement(body, "cb", pageside=column.group("page_num"), side=column.group("side"))
    cb.text = row["Griechisch"]
  if page:
    pb = etree.SubElement(body, "pb", n=page.group("pg"))
    pb.text = row["Griechisch"]
  if not column and not page:
    marg_note = etree.SubElement(body, "note")
    marg_note.text = row["Griechisch"]

In [26]:
def verb_gram():
  form = etree.SubElement(entry, "form", type="inflected")
  gramGrp = etree.SubElement(form, "gramGrp")
  gram = etree.SubElement(gramGrp, "gram", type="pos", value="verb")
  gram = etree.SubElement(gramGrp, "gram", type="tense", value="aorist")
  orth = etree.SubElement(form, "orth")
  orth.text = verb.group("aorist")
  form = etree.SubElement(entry, "form", type="inflected")
  gramGrp = etree.SubElement(form, "gramGrp")
  gram = etree.SubElement(gramGrp, "gram", type="pos", value="verb")
  gram = etree.SubElement(gramGrp, "gram", type="tense", value="perfect")
  orth = etree.SubElement(form, "orth")
  orth.text = verb.group("perfect")

In [107]:
def grammar(MSA):
  noun = re.search("(ὁ|ἡ|τὸ)", MSA)
  nouns = re.search("(οἱ|αἱ|τὰ)", MSA)
  verb = re.search("((\\?μ),?\s(?P<aorist>.*?))?\s((π),?\s(?P<perfect>.*))?", MSA)
  adj = re.search("((.*?),\s?)?(.?ὁ καὶ ἡ.?)", MSA)
  just_noun = re.search("(.*?),\s(ὁ|ἡ|τὸ)", MSA)
  genitive_noun = re.search("(?P<first>.*?)\s(καὶ)\s(?P<second>.*?),\s?(?P<gen>ὁ|ἡ|τὸ)", MSA)
  def gen_check():
    if re.search("ὁ", MSA):
      gram = etree.SubElement(gramGrp, "gram", type="gen", value="m")
      gram.text = "ὁ"
    if re.search("ἡ", MSA):
      gram = etree.SubElement(gramGrp, "gram", type="gen", value="f")
      gram.text = "ἡ"
    if re.search("τὸ", MSA):
      gram = etree.SubElement(gramGrp, "gram", type="gen", value="n")
      gram.text = "τὸ"
  def verb_gram():
    form = etree.SubElement(entry, "form", type="inflected")
    gramGrp = etree.SubElement(form, "gramGrp")
    gram = etree.SubElement(gramGrp, "gram", type="pos", value="verb")
    gram = etree.SubElement(gramGrp, "gram", type="tense", value="aorist")
    orth = etree.SubElement(form, "orth")
    orth.text = verb.group("aorist")
    form = etree.SubElement(entry, "form", type="inflected")
    gramGrp = etree.SubElement(form, "gramGrp")
    gram = etree.SubElement(gramGrp, "gram", type="pos", value="verb")
    gram = etree.SubElement(gramGrp, "gram", type="tense", value="perfect")
    orth = etree.SubElement(form, "orth")
    orth.text = verb.group("perfect")
  if type(MSA) != float and  type(slovar["GLemma"][index+1]) != float:
      if noun:
        form = etree.SubElement(entry, "form", type="inflected")
        gramGrp = etree.SubElement(form, "gramGrp")
        gram = etree.SubElement(gramGrp, "gram", type="number", value="sg")
        gram = etree.SubElement(gramGrp, "gram", type="pos", value="noun")
        if re.search("^(ὁ|ἡ|τὸ)$", MSA):
          gen_check()
        if just_noun:
          gram = etree.SubElement(gramGrp, "gram", type="case", value="genitiv")
          orth = etree.SubElement(form, "orth", extent="part")
          orth.text = just_noun.group(1)
          gen_check()
        if genitive_noun:
          gram = etree.SubElement(gramGrp, "gram", type="case", value="genitiv")
          form_var1 = etree.SubElement(form, "form", type="variant", n=1)
          orth = etree.SubElement(form_var1, "orth", extent="part")
          orth.text = genitive_noun.group("first")
          form_var2 = etree.SubElement(form, "form", type="variant", n=2)
          orth = etree.SubElement(form_var2, "orth", extent="part")
          orth.text = genitive_noun.group("second")
          gen_check()
      elif nouns:
        form = etree.SubElement(entry, "form", type="inflected")
        gramGrp = etree.SubElement(form, "gramGrp")
        gram = etree.SubElement(gramGrp, "gram", type="pos", value="noun")
        gram = etree.SubElement(gramGrp, "gram", type="number", value="pl")
        if re.search("οἱ", MSA):
          gram = etree.SubElement(gramGrp, "gram", type="gen", value="m")
          gram.text = "οἱ"
        if re.search("αἱ", MSA):
          gram = etree.SubElement(gramGrp, "gram", type="gen", value="f")
          gram.text = "αἱ"
        if re.search("τὰ", MSA):
          gram = etree.SubElement(gramGrp, "gram", type="gen", value="n")
          gram.text = "τὰ"
      elif adj:
        form = etree.SubElement(entry, "form", type="inflected")
        gramGrp = etree.SubElement(form, "gramGrp")
        gram = etree.SubElement(gramGrp, "gram", type="pos", value="adj")
        gram = etree.SubElement(gramGrp, "gram", type="number", value="sg")
        gram = etree.SubElement(gramGrp, "gram", type="gen", value="m")
        gram = etree.SubElement(gramGrp, "gram", type="gen", value="f")
        gram = etree.SubElement(gramGrp, "gram", type="case", value="genitiv")
        orth = etree.SubElement(form, "orth", extent="part")
        orth.text = MSA  
        print(MSA)
      elif MSA == "ῶ" and type(slovar["GMSA"][index+1]) != float:
        slovar["GMSA"][index] = slovar["GMSA"][index]+" "+ slovar["GMSA"][index+1]
        if type(slovar["GMSA"][index+2]) != float and type(slovar["Griechisch"][index+2]) == float:
          slovar["GMSA"][index] = slovar["GMSA"][index]+" "+slovar["GMSA"][index+2]
        form = etree.SubElement(entry, "form", type="variant")
        gramGrp = etree.SubElement(form, "gramGrp")
        gram = etree.SubElement(gramGrp, "gram", type="pos", value="verb")
        gram = etree.SubElement(gramGrp, "gram")
        gram.text = "ῶ"
        verb_gram()
      elif verb:
        if type(slovar["GMSA"][index+1]) != float and type(slovar["Griechisch"][index+1]) == float:
          slovar["GMSA"][index] = slovar["GMSA"][index]+" "+slovar["GMSA"][index+1]
          verb_gram()
      elif MSA == "ῶ":
        form = etree.SubElement(entry, "form", type="variant")
        gramGrp = etree.SubElement(form, "gramGrp")
        gram = etree.SubElement(gramGrp, "gram")
        gram.text = "ῶ"
      else:
        form = etree.SubElement(entry, "form", type="inflected")
        gramGrp = etree.SubElement(form, "gramGrp")
        gram = etree.SubElement(gramGrp, "gram")
        orth = etree.SubElement(form, "orth")
        orth.text = MSA 

In [28]:
 def slavic_latin_translation():
   cit = etree.SubElement(sense_greek, "cit", type="translationEquivalent", xml_lang = "chu")
   form = etree.SubElement(cit, "form",  xml_id=row["SÜbersetzung"].strip(","))
   orth = etree.SubElement(form, "orth")
   orth.text = row["SÜbersetzung"].strip(",")
   if type(row["Lateinisch"]) != float and row["Lateinisch"] != "-":
        sense = etree.SubElement(cit, "sense", xml_id=row["SÜbersetzung"].strip(",")+".1")
        cit = etree.SubElement(sense, "cit", type="translationEquivalent", xml_lang = "lat")
        form = etree.SubElement(cit, "form", type="hyperlemma", xml_id=row["Lateinisch"])
        orth = etree.SubElement(form, "orth")
        if type(slovar["SÜbersetzung"][index+1]) == float \
        and type(slovar["Lateinisch"][index+1]) != float and slovar["Lateinisch"][index+1] != "-":
          orth.text = row["Lateinisch"]+" "+slovar["Lateinisch"][index+1]
        else:
          orth.text = row["Lateinisch"]
   if type(row["LLemma"]) != float and row["LLemma"] != "-":
    form = etree.SubElement(cit, "form", type="lemma", xml_id=row["LLemma"])
    orth = etree.SubElement(form, "orth")
    orth.text = row["LLemma"]

In [29]:
def several_slavic_equivalents_check(Slavic, Latin, LatLemma):
  global index
  while type(slovar["GLemma"][index+1]) == float:
    if type(Slavic[index+1]) != float:
      cit = etree.SubElement(sense_greek, "cit", type="translationEquivalent", xml_lang = "chu")
      form = etree.SubElement(cit, "form",  xml_id=Slavic[index+1].strip(","))
      orth = etree.SubElement(form, "orth")
      orth.text = Slavic[index+1].strip(",")
      if type(Latin[index+1]) != float and Latin[index+1] != "-":
        sense = etree.SubElement(cit, "sense", xml_id=Slavic[index+1]+".1")
        cit = etree.SubElement(sense, "cit", type="translationEquivalent", xml_lang = "lat")
        form = etree.SubElement(cit, "form", type="hyperlemma", xml_id=Latin[index+1])
        orth = etree.SubElement(form, "orth")
        if type(Slavic[index+2]) == float and type(Latin[index+2]) != float and Latin[index+2] != "-": 
          orth.text = Latin[index+1]+" "+Latin[index+2]
          if type(Slavic[index+3]) == float \
          and type(Latin[index+3]) != float and Latin[index+3] != "-":
            orth.text = Latin[index+1]+" "+Latin[index+2]+" "+Latin[index+3]
        else:
          orth.text = Latin[index+1]
      if type(LatLemma[index+1]) != float and LatLemma[index+1] != "-":
        if type(Latin[index+1]) == float or Latin[index+1] == "-":
          sense = etree.SubElement(cit, "sense", xml_id=Slavic[index+1].strip(",")+".1")
          cit = etree.SubElement(sense, "cit", type="translationEquivalent", xml_lang = "lat")
        form = etree.SubElement(cit, "form", type="lemma", xml_id=LatLemma[index+1])
        orth = etree.SubElement(form, "orth")
        if type(Slavic[index+2]) == float and type(LatLemma[index+2]) != float and LatLemma[index+2] != "-": 
          orth.text = LatLemma[index+1]+" "+LatLemma[index+2]
          if type(Slavic[index+3]) == float \
          and type(LatLemma[index+3]) != float and LatLemma[index+3] != "-":
            orth.text = LatLemma[index+1]+" "+LatLemma[index+2]+" "+LatLemma[index+3]
        else:
          orth.text = LatLemma[index+1]
    index+= 1

In [105]:
def semantischer_kommentar():
  global index
  if type(row["GLemma"]) == float and type(slovar["SÜbersetzung"][index-1]) != float:
    index-= 1
    while type(slovar["GLemma"][index-1]) == float and type(slovar["SÜbersetzung"][index]) != float:
      row["GLemma"] = slovar["GLemma"][index-1]
      index-= 1
    row["GLemma"] = slovar["GLemma"][index-1]
  note_in_entry = re.search("(\[.*\])", row["SemantK"])
  cross_reference_mention = re.search("^(тожде єже|тожде|таже)\s(.*)", row["SemantK"])
  sense_note = etree.SubElement(entry, "sense", xml_id=row["GLemma"]+".1")
  note = etree.SubElement(sense_note, "note")
  if row["SemantK"] == "тожде":
    xr = etree.SubElement(note, "xr", type="synonymy")
    while type(slovar["GLemma"][index-1]) == float:
      index-= 1
    ref = etree.SubElement(xr, "ref", type="entry", target="#"+slovar["GLemma"][index-1])
    ref.text = row["SemantK"]
  if cross_reference_mention:
    xr = etree.SubElement(note, "xr", type="synonymy")
    lbl = etree.SubElement(xr, "lbl")
    lbl.text = cross_reference_mention.group(1)
    ref = etree.SubElement(xr, "ref", type="entry", target="#"+cross_reference_mention.group(2))
    ref.text = cross_reference_mention.group(2)
  if not row["SemantK"] == "тожде" and not cross_reference_mention:
    note.text = row["SemantK"]
  if note_in_entry:
    note_editorial = etree.SubElement(note, "note", type="editor")
    note_editorial.text = note_in_entry.group(1)


In [49]:
generate_xml()
ElementTree(root).write('dictionary_TEI.xml', pretty_print=True, xml_declaration = True, encoding='UTF-8', standalone = None)

NameError: ignored

In [ ]:
#